# Matrix product state representation for ground states

$
\require{physics}
\def\bm{\boldsymbol}
$

## Introduction

Clearly, God did not conceive the universe as a point because God would not be
able to take the outer product of two points to obtain vectors.
So in the beginning, there were at least two vectors, and God took the outer
product to make higher order tensors, the inner product to make lower order 
tensors: these were the binary operators on tensors.
God defined these tensors over a field and also used the transpose and inverse.
We believe God may have saved us because God invented index notation 
so that any of this could make sense.
Therefore tensors, their fields, their classes, their algebras, and their calculus
is the only way that it is convenient to express the geometry of the universe.

Sometimes smart people find ways to represent large tensors by smaller tensors,
and this assignment is originated in this observation.
The goal is to rewrite a tensor of dimension $d^L$ which describes the state
of a qu$d$it chain of known $L$ength as a matrix product.
We do so using the SVD and Schmidt decomposition techniques explored earlier.

Other projects seem pretty useful to look into for helpful
[resources](https://tensornetwork.org/) and
[software](https://github.com/google/TensorNetwork).
This is a [useful paper](https://arxiv.org/abs/1008.3477) cited in the
assignment on the subject.

## MPS representation

One asks how you actually do this task efficiently, which comes down to knowing
how to represent the MPS wavefunction in memory.
Choose a positive integer value for $d$ and $L$.
The representation is of the form
$$
    \psi_{\sigma_1, \dots, \sigma_L} = 
        \sum_{\alpha_0, \dots, \alpha_L}
        \left( \prod_{i=0}^{L-1} A_{\alpha_i \alpha_{i+1}}^{\sigma_{i+1}} \right)
$$
where
\begin{align}
    \dim(\alpha_i) \leq 
        \begin{cases}
            d^i
                &\qq{i \leq L/2}
            \\\\
            d^{L-i}
                &\qq{i > L/2}
        \end{cases}
    .
\end{align}
The form of this dimensionality is due to the way the mps representation is 
constructed: a tensor is reshaped and has its SVD taken many times,
each new time after another reshaping and SVD.
Essentially these dimensions are the maximal ranks of a matrix as it is reshaped
from $(1, d^L)$ to $(d^L, 1)$ when exchanging rows to columns $d$ at a time.
Now an approximation scheme for MPS is a map:
\begin{align}
    r(i) : \{i\} \to \{1, \dots, \max(1, \min(d r(i-1), \dim(\alpha_i))\}
\end{align}
with $r(0) = 1$
that specifies the number of rows retained at each bond index.
This is a finite but large function space, and of the many approximation schemes
it often makes sense to choose a simple one, such as
$r(i) = \min(\chi, \dim(\alpha_i))$ for constant $\chi$.

### Scaling

The scaling of storage requirements as a function of the approximation scheme
can be calculated succinctly as $d \sum_i r(i) r(i+1)$.
Taking $r=\dim(\alpha_i)$ makes for an inefficient full representation of the
because of the $d^L$ scaling for the individual matrices.
For what truncation ranks $r$ is MPS an efficient storage scheme?
We will test the accuracy of the scheme later.

## Program

- $d=2, r=\chi=1$ ($r=1$ is a special case where the storage format can be optimized)
- For $h \in \{ 1, 5/4 \}$
- Take the ground state in the open system at large L
- Compute the MPS approximation of the wavefunction, varying the bond length $k$
- Calculate the actual reduction in storage space (np.size)
- Contract the indices of the tensors of MPS to obtain exponentially large
$\ket{\tilde{\psi}_{gs} (k)}$.
- Compute the overlap $\braket{\tilde{\psi}\_{gs} (k)}{\psi\_{gs}}$

### Efficient calculations with MPS

- Use MPS to calculate
$E(k) = \ev{H}{\tilde{\psi}\_{gs} (k)} / \braket{\tilde{\psi}\_{gs} (k)}$
- Compute the same correlation functions as in Assignment 1 at both values of 
the order parameter and study the convergence in $k$.

In [ ]:
from itertools import product 

import numpy as np
import pandas as pd
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import basis, tfim, tests, tensor

In [ ]:
%%time
measurements = {
    'bc': [],
    'L' : [],
    'h' : [],
    'k' : [],
    'N' : [],
    'nm': [],
    'E' : [],
    'Ek': [],
    'ip': [],
    'Cz': [],
    'Mz': [],
}

for oper_params in tests.tfim_sweep(
    L = [16],
    h = [1, 1.2],
    bc= ['o'],
):
    job = dict(
        oper=tfim.z.H_sparse,
        oper_params=oper_params,
        solver=sla.eigsh,
        solver_params={ 
            'k' : 6, 
            'which' : 'BE',
        },
    )
    evals, evecs = data.jobs.obtain(**job)
    
    # construct local operators
    sx = np.array([[0, 1], [1, 0]], dtype='float64')
    sz = np.array([[1, 0], [0, -1]], dtype='float64')
        
    C = tensor.mpo(oper_params['L'], d=2)
    C[0] = sz
    C[oper_params['L'] // 2] = sz
    
    # Construct operators which are sums of local operators
    H = []
    M = []
    
    # Hamiltonian
    ## z terms
    for i in range(oper_params['L'] - 1 + (oper_params['bc'] == 'c')):
        H.append(tensor.mpo(oper_params['L'], d=2))
        H[-1][i] = -sz
        H[-1][(i+1) % oper_params['L']] = sz
    ## x terms
    for i in range(oper_params['L']):
        H.append(tensor.mpo(oper_params['L'], d=2))
        H[-1][i] = -oper_params['h'] * sx
        
    # Magnetization
    for i, j in product(np.arange(oper_params['L']), repeat=2):
        M.append(tensor.mpo(oper_params['L'], d=2))
        M[-1][i] = sz / (oper_params['L'] ** 2)
        M[-1][j] = sz

    chi_max = 20
    def rank (i, x=chi_max):
        """Define an approximation scheme given by a maximal bond length."""
        return max(1, min(x, tensor.mps.dim_mps(i, L=oper_params['L'], d=2)))

    A = tensor.mps(evecs[:, 0], rank, L=oper_params['L'], d=2)
    for i in range(chi_max - 1):
        def rank (i, x=chi_max-i):
            """Define an approximation scheme given by a maximal bond length."""
            return max(1, min(x, tensor.mps.dim_mps(i, L=oper_params['L'], d=2)))
        A.lower_rank(rank)
        measurements['bc'].append(
            oper_params['bc']
        )
        measurements['L'].append(
            oper_params['L']
        )
        measurements['h'].append(
            oper_params['h']
        )
        measurements['k'].append(
            chi_max - i
        )
        measurements['N'].append(
            A.size()
        )
        measurements['nm'].append(
            A.inner(A)
        )
        measurements['E'].append(
            evals[0]
        )
        measurements['ip'].append(
            np.inner(A.v, A.contract_bonds()) / np.sqrt(measurements['nm'][-1])
        )
        measurements['Ek'].append(
            sum(A.expval(e) for e in H) / measurements['nm'][-1]
        )
        measurements['Cz'].append(
            A.expval(C) / measurements['nm'][-1]
        )
        measurements['Mz'].append(
            sum(A.expval(e) for e in M) / measurements['nm'][-1]
        )
        
df = pd.DataFrame(measurements)

In [ ]:
%%capture plot

myplots = ['Ek', 'N', 'ip', 'Cz', 'Mz']
ncol = 2
nrow = len(myplots) // 2 + len(myplots) % 2

fig, axes = plt.subplots(nrow, ncol)

for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if (ncol * i + j) < len(myplots):
            for h in [1, 1.2]:
                ax.plot(
                    df[df.h == h]['k'].values,
                    df[df.h == h][myplots[i * ncol + j]].values,
                    label='h=' + str(h),
                )
            ax.set_xlabel('k')
            ax.set_ylabel(myplots[i * ncol + j])
            handles, labels = ax.get_legend_handles_labels()
        else:
            ax.set_axis_off()
            ax.legend(handles, labels, loc='center')
    fig.tight_layout()

In [ ]:
%%capture corr
def rank (i, x=chi_max):
    """Define an approximation scheme given by a maximal bond length."""
    return max(1, min(x, basis.mps.dim_mps(i, L=oper_params['L'], d=2)))

A = basis.mps.my_mps(evecs[:, 0], rank, L=oper_params['L'], d=2)

C = np.empty(oper_params['L'], dtype='object')
x = [0]
y = [1]
C[0] = sz
for i in range(1, oper_params['L']):
    C[i] = sz
    x.append(i)
    y.append(A.expval(C) / measurements['nm'][-1])
    C[i] = None
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlabel('r')
ax.set_ylabel('$C^{zz}_r$')
plt.show()

## Results

### Accuracy

I first wanted to show some of the numerical results in decimal form so that
I have an opportunity to explain my abbreviations, and to show that the
calculations are relatively accurate

In [ ]:
df.head()

In [ ]:
plot.show()

In [ ]:
corr.show()

## Discussion